In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

In [ ]:
canada_rate_file = "Resources/Exchangerates_Canada.csv"
china_rate_file = "Resources/Exchangerates_China.csv"
euro_rate_file = "Resources/Exchangerates_Euro.csv"
hongkong_rate_file = "Resources/Exchangerates_HongKong.csv"
india_rate_file = "Resources/Exchangerates_India.csv"
japan_rate_file = "Resources/Exchangerates_Japan.csv"
korea_rate_file = "Resources/Exchangerates_Korea.csv"
southafrica_rate_file = "Resources/Exchangerates_SouthAfrica.csv"
swiss_rate_file = "Resources/Exchangerates_Swiss.csv"

index_data_file = "Resources/indexData.csv"
info_data_file = "Resources/infoData.csv"

In [ ]:
canada_df = pd.read_csv(canada_rate_file)
china_df = pd.read_csv(china_rate_file)
euro_df = pd.read_csv(euro_rate_file)
hongkong_df = pd.read_csv(hongkong_rate_file)
india_df = pd.read_csv(india_rate_file)
japan_df = pd.read_csv(japan_rate_file)
korea_df = pd.read_csv(korea_rate_file)
southafrica_df = pd.read_csv(southafrica_rate_file)
swiss_df = pd.read_csv(swiss_rate_file)

data_df = pd.read_csv(index_data_file)
info_df = pd.read_csv(info_data_file)

In [ ]:
data_df

In [ ]:
# keep only stock exchange data from year 2000 and later
data_df = data_df.loc[data_df['Date']>= '2000-01-01']

#remove taiwan data as we do not have exchange rates for this stock exchange.
data_df = data_df.loc[data_df['Index']!= 'TWII']

#drop unecessary column
data_df.drop(columns='Adj Close',inplace= True)

# stock exchange data with null values
len(data_df)

In [ ]:
data_df.dropna(inplace=True)
len(data_df)

In [ ]:
#convert date column string from exchnage rate data frames into date objects to macth data_df

canada_df['Date']= pd.to_datetime(canada_df['Date'])
china_df['Date']= pd.to_datetime(china_df['Date'])
euro_df['Date']= pd.to_datetime(euro_df['Date'])
hongkong_df['Date']= pd.to_datetime(hongkong_df['Date'])
india_df['Date']= pd.to_datetime(india_df['Date'])
japan_df['Date']= pd.to_datetime(japan_df['Date'])
korea_df['Date']= pd.to_datetime(korea_df['Date'])
southafrica_df['Date']= pd.to_datetime(southafrica_df['Date'])
swiss_df['Date']= pd.to_datetime(swiss_df['Date'])

data_df['Date']= pd.to_datetime(data_df['Date'])

In [ ]:
#add column for currency type to merge on later
canada_df['Currency']= 'CAD'
china_df['Currency']= 'CNY'
euro_df['Currency']= 'EUR'
hongkong_df['Currency']= 'HKD'
india_df['Currency']= 'INR'
japan_df['Currency']= 'JPY'
korea_df['Currency']= 'KRW'
southafrica_df['Currency']= 'ZAR'
swiss_df['Currency']= 'CHF'

In [ ]:
#rename column
canada_df.rename(columns = {'CAD':'exchange_rate'}, inplace = True)
china_df.rename(columns = {'CNY':'exchange_rate'}, inplace = True)
euro_df.rename(columns = {'EUR':'exchange_rate'}, inplace = True)
hongkong_df.rename(columns = {'HKD':'exchange_rate'}, inplace = True)
india_df.rename(columns = {'INR':'exchange_rate'}, inplace = True)
japan_df.rename(columns = {'JPY':'exchange_rate'}, inplace = True)
korea_df.rename(columns = {'KRW':'exchange_rate'}, inplace = True)
southafrica_df.rename(columns = {'ZAR':'exchange_rate'}, inplace = True)
swiss_df.rename(columns = {'CHF':'exchange_rate'}, inplace = True)

In [ ]:
#concatenate all exchange rate data frames into one
exchange_rate_df = pd.concat([canada_df, china_df, euro_df, hongkong_df, india_df, japan_df, korea_df, southafrica_df, swiss_df])
exchange_rate_df

In [ ]:
exchange_rate_df.dtypes

In [ ]:
exchange_rate_df['exchange_rate'] = exchange_rate_df['exchange_rate'].replace(',', '',regex=True)

In [ ]:
exchange_rate_df['exchange_rate'] = exchange_rate_df['exchange_rate'].astype(float)
exchange_rate_df

In [ ]:
# keep only exchange rate data from year 2000 and later
exchange_rate_df = exchange_rate_df.loc[exchange_rate_df['Date']>= '2000-01-01']


exchange_rate_df

In [ ]:
# merge info df into data df
merge_df = pd.merge(data_df,info_df,on='Index',how='left')
merge_df

In [ ]:
stock_exchange_df = pd.merge(merge_df,exchange_rate_df, on=['Currency','Date'], how='left')
stock_exchange_df

In [ ]:
#check for null values
df_null = stock_exchange_df.loc[stock_exchange_df['exchange_rate'].isnull()]
df_null['Index'].unique()

In [ ]:
#There is no exchange rate values for USD sice that is our base currency, solution: fill all null values with 1.
stock_exchange_df['exchange_rate'].fillna(1, inplace = True)
stock_exchange_df['USD'].fillna(1, inplace = True)
stock_exchange_df

In [ ]:
#Do conversion of columns to USD
stock_exchange_df['Open_USD']=stock_exchange_df['Open']/stock_exchange_df['exchange_rate']
stock_exchange_df['High_USD']=stock_exchange_df['High']/stock_exchange_df['exchange_rate']
stock_exchange_df['Low_USD']=stock_exchange_df['Low']/stock_exchange_df['exchange_rate']
stock_exchange_df['Close_USD']=stock_exchange_df['Close']/stock_exchange_df['exchange_rate']
stock_exchange_df

In [ ]:
engine = create_engine('sqlite:///stock_exchange_db.sqlite', echo=True)
sqlite_connection = engine.connect()

In [ ]:
sqlite_table = "stock_exchange"
stock_exchange_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

In [ ]:
sqlite_connection.close()